In [7]:
import re
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor
from threading import Lock

class NhatotMasterCrawler:
    def __init__(self, headless=False):
        self.base_url = "https://www.nhatot.com/tham-khao-gia-mua-ban-nha-dat-{}-tp-ho-chi-minh"
        self.headless = headless
        self.print_lock = Lock()

    def _init_driver(self):
        chrome_options = Options()
        if self.headless:
            chrome_options.add_argument('--headless=new')

        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        chrome_options.add_argument('--window-size=1366,768')
        # Thêm các option giúp tránh bị block và render tốt hơn
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        return driver

    def extract_numbers(self, text: str):
        # Loại bỏ dấu chấm ngăn cách hàng nghìn, đổi dấu phẩy thành dấu chấm thập phân
        cleaned = text.replace('.', '').replace(',', '.')
        matches = re.findall(r"[-+]?\d*\.\d+|\d+", cleaned)
        return [float(m) for m in matches]

    def get_data_from_section(self, section_soup):
        price_values = []
        elements = section_soup.find_all(['div', 'span', 'text'])

        for el in elements:
            txt = el.get_text(strip=True)
            # Regex kiểm tra chuỗi chỉ chứa số và ký tự phân tách
            if re.match(r'^[\d.,]+$', txt):
                val = self.extract_numbers(txt)
                if val: price_values.append(val[0])

        # Lọc số trong khoảng giá nhà đất hợp lý
        valid_prices = sorted(list(set([p for p in price_values if 10 < p < 3000])))

        avg, low, high = None, None, None
        if len(valid_prices) >= 3:
            low = valid_prices[0] # Giá thấp nhất bên trái
            high = valid_prices[-1] # Giá cao nhất bên phải
            # Lấy số nằm giữa (giá trung bình lơ lửng trên bar)
            remaining = [p for p in valid_prices if p != low and p != high]
            avg = remaining[0] if remaining else (low + high) / 2
        elif len(valid_prices) == 1:
            avg = valid_prices[0]

        return avg, low, high

    def crawl_single_district(self, district_tuple):
        district_name, slug = district_tuple
        driver = self._init_driver()
        results = []

        try:
            url = self.base_url.format(slug)
            driver.get(url)

            # Đợi trang tải sơ bộ
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # Cuộn trang từ từ để biểu đồ load (quan trọng)
            for i in range(3):
                driver.execute_script(f"window.scrollTo(0, {300 * (i+1)});")
                time.sleep(1.5)

            # Đợi cụ thể một con số giá xuất hiện trên chart
            time.sleep(5)

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            sections = [
                {"title": "Giá Nhà ở", "type": "nha_o", "hem": 0, "mt": 0},
                {"title": "Giá Nhà mặt phố", "type": "nha_mat_pho", "hem": 0, "mt": 1},
                {"title": "Giá Nhà ngõ, hẻm", "type": "nha_ngõ_hẻm", "hem": 1, "mt": 0}
            ]

            for sec in sections:
                # Tìm header chứa tiêu đề loại nhà
                header = soup.find(['h2', 'h3', 'h4'], string=re.compile(sec["title"], re.I))
                if header:
                    container = header.find_parent('div')
                    avg, low, high = self.get_data_from_section(container)

                    if avg or low: # Chỉ add nếu lấy được dữ liệu
                        results.append({
                            "district": district_name,
                            "type_house": sec["type"],
                            "is_hem": sec["hem"],
                            "is_mattien": sec["mt"],
                            "price_per_m2_avg": avg,
                            "price_per_m2_low_avg": low,
                            "price_per_m2_high_avg": high
                        })

            with self.print_lock:
                status = "✅" if results else "⚠️ Không lấy được data"
                print(f"{status}: {district_name}")

        except Exception as e:
            with self.print_lock:
                print(f"❌ Lỗi tại {district_name}: {str(e)}")
        finally:
            driver.quit()

        return results

    def run(self, districts_dict, workers=2): # Giảm worker xuống 2 để tránh lag máy khi mở Chrome
        print(f"🚀 Bắt đầu quét đa luồng ({workers} luồng - HIỆN TRÌNH DUYỆT)...")
        all_data = []

        with ThreadPoolExecutor(max_workers=workers) as executor:
            future_results = list(executor.map(self.crawl_single_district, districts_dict.items()))

            for res in future_results:
                if res:
                    all_data.extend(res)

        return pd.DataFrame(all_data)

if __name__ == "__main__":
    districts = {
        "Quận 1": "quan-1",
        "Quận 3": "quan-3",
        "Thành phố Thủ Đức": "thanh-pho-thu-duc",
        "Quận 4": "quan-4",
        "Quận 5": "quan-5",
        "Quận 6": "quan-6",
        "Quận 7": "quan-7",
        "Quận 8": "quan-8",
        "Quận 10": "quan-10",
        "Quận 11": "quan-11",
        "Quận 12": "quan-12",
        "Quận Bình Tân": "quan-binh-tan",
        "Quận Bình Thạnh": "quan-binh-thanh",
        "Quận Gò Vấp": "quan-go-vap",
        "Quận Phú Nhuận": "quan-phu-nhuan",
        "Quận Tân Bình": "quan-tan-binh",
        "Quận Tân Phú": "quan-tan-phu",
        "Huyện Bình Chánh": "huyen-binh-chanh",
        "Huyện Cần Giờ": "huyen-can-gio",
        "Huyện Củ Chi": "huyen-cu-chi",
        "Huyện Hóc Môn": "huyen-hoc-mon"
    }

    # Tôi để mặc định headless=False để bạn thấy Chrome chạy.
    # workers=2 để máy không bị quá tải khi mở nhiều Chrome cùng lúc.
    crawler = NhatotMasterCrawler(headless=False)
    df_final = crawler.run(districts, workers=2)

    if not df_final.empty:
        print("\n" + "="*50)
        print("DỮ LIỆU ĐÃ THU THẬP")
        print("="*50)
        print(df_final.to_string(index=False))
        df_final.to_csv("Bench.csv", index=False, encoding="utf-8-sig")
        print(f"\n💾 Lưu thành công {len(df_final)} dòng.")
    else:
        print("\n❌ Vẫn không lấy được dữ liệu. Hãy kiểm tra kết nối mạng hoặc thử tăng time.sleep.")

🚀 Bắt đầu quét đa luồng (2 luồng - HIỆN TRÌNH DUYỆT)...
✅: Quận 3
✅: Quận 1
✅: Thành phố Thủ Đức
✅: Quận 4
✅: Quận 5
✅: Quận 6
✅: Quận 7
✅: Quận 8
✅: Quận 10
✅: Quận 11
✅: Quận 12
✅: Quận Bình Tân
✅: Quận Bình Thạnh
✅: Quận Gò Vấp
✅: Quận Tân Bình
✅: Quận Phú Nhuận
✅: Quận Tân Phú
✅: Huyện Bình Chánh
✅: Huyện Cần Giờ
✅: Huyện Củ Chi
✅: Huyện Hóc Môn

DỮ LIỆU ĐÃ THU THẬP
         district  type_house  is_hem  is_mattien  price_per_m2_avg  price_per_m2_low_avg  price_per_m2_high_avg
           Quận 1       nha_o       0           0            366.67                261.38                 727.27
           Quận 1 nha_mat_pho       0           1            450.82                347.22                 882.35
           Quận 1 nha_ngõ_hẻm       1           0            230.00                185.71                 320.00
           Quận 3       nha_o       0           0            266.67                179.10                 541.36
           Quận 3 nha_mat_pho       0           1            4